In [1]:
#V1.1

import requests
import json
import openpyxl
from openpyxl.styles import PatternFill, Alignment, Border, Side
from tqdm import tqdm
from datetime import datetime
import urllib3
import pytz
from dotenv import load_dotenv
import os
from collections import defaultdict
import sys



def get_token(username, password):
    login_path = "https://edm-delman.apps.binus.edu/analytic/login"

    payload = json.dumps({
        "username": username,
        "password": password
    })
    headers = {
    'Content-Type': 'application/json'
    }

    token = requests.post(
        login_path, 
        headers=headers, 
        data=payload, 
        verify=False
    ).headers["Authorization"]
    
    return token

def convert_timezone_to_gmt7(datetime_str):
    # Define timezones
    utc_timezone = pytz.timezone('Etc/GMT')
    gmt7_timezone = pytz.timezone('Asia/Bangkok')  # GMT+0700 is Indochina Time (ICT)
    
    # Parse the datetime string
    dt = datetime.strptime(datetime_str, '%a, %d %b %Y, %H:%M:%S GMT+0000')
    
    # Localize to UTC timezone
    utc_dt = utc_timezone.localize(dt)
    
    # Convert to GMT+0700 (ICT)
    gmt7_dt = utc_dt.astimezone(gmt7_timezone)
    
    return gmt7_dt.strftime('%a, %d %b %Y, %H:%M:%S GMT+0700')


def generate_excel(token, excel, proj_file="projects (2).json"):
    if not os.path.exists(proj_file):
        print(f"Error: {proj_file} does not exist.")
        return
    
    with open(proj_file) as f:
        proj_dict = json.load(f)
    
    proj_url = "https://edm-delman.apps.binus.edu/analytic/projects/"

    explored = defaultdict(lambda:None)
    stat = ["SUCCESS", None, "CREATED", "UPSTREAM FAILED"]

    for i, proj in tqdm(enumerate(proj_dict), total=len(proj_dict)):
        error_node = None
        status = ""
        if proj["id"] != None:
            if explored[proj["id"]] is not None:
                note, status = explored[proj["id"]]
                _, _ = excel.write_line(proj, i+2, error_note=note, status=status)
                continue
            
            response = requests.get(
                proj_url+proj["id"]+"/monitoring?page_size=8&page=0",
                headers= {'Authorization': token},
                verify=False
            )
            
            # Check the sync date
            data = json.loads(response.content)['data']
            if data:
                for entry in data:
                    if entry.get('started_at'):  # Check if 'started_at' exists and is not None
                        date_sync = entry['started_at']
                        gmt7_datetime = convert_timezone_to_gmt7(date_sync)
                        datetime_obj = datetime.strptime(gmt7_datetime, "%a, %d %b %Y, %H:%M:%S %Z%z")
                        date_only = datetime_obj.date()
                        current_date = datetime.now().date()
                        # Perform further operations with date_sync if needed
                        break  # Exit the loop once a valid 'started_at' is found
    
                if  date_only >= current_date:
                    response2 = requests.get(
                        proj_url+proj["id"],
                        headers= {'Authorization': token},
                        verify=False
                        )
                    
                    try:
                        nodes = json.loads(response2.content)['data']['nodes']
                    except json.JSONDecodeError as e:
                        print(f"JSON decode error: {e}")
                        print(response2.content)
                        nodes = []
                    error_node = [n for n in nodes if n['status'] not in stat or n['export_status'] not in stat]
                else:
                    status="Not Synced"
            
        note, status = excel.write_line(proj, i+2, error_node, status=status)
        explored[proj["id"]] = (note, status)
        
    excel.save('summary_job_dwh.xlsx')

class ExcelWriter():
    def __init__(self):
        self.today = datetime.today().date().strftime("%d-%b-%y")
        self.wb = openpyxl.Workbook()
        self.sheet = self.wb.active
        self.side = Side(border_style='thin', color='000000')
        header_border = Side(border_style='thin', color='000000')
        self.sheet["A1"] = self.today
        self.sheet.merge_cells('A1:L1')
        self.sheet["A1"].fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')
        self.sheet["A1"].alignment = Alignment(horizontal='center', vertical='center')
        self.sheet["A1"].border = Border(left=header_border, right=header_border, top=header_border, bottom=header_border)

        self.sheet.column_dimensions['B'].width = 11.43
        self.sheet.column_dimensions['C'].width = 11.43
        self.sheet.column_dimensions['F'].width = 54.57
        self.sheet.column_dimensions['G'].width = 56.71
        self.sheet.column_dimensions['H'].width = 12.14
        self.sheet.column_dimensions['I'].width = 16
        self.sheet.column_dimensions['J'].width = 74.14
        
        self.colors = {
            "Success": PatternFill(start_color='C6EFCE', end_color='C6EFCE', fill_type='solid'),
            "Failed": PatternFill(start_color='FFC7CE', end_color='FFC7CE', fill_type='solid'),
            "Not Synced": PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid'),
            "": PatternFill(start_color='FFFFFF', end_color='FFFFFF', fill_type='solid')
        }

    def write_line(self, proj, row_id, error_nodes=None, error_note=None, status=""):
        if error_note == None:
            error_note = ""
            if error_nodes is None:
                pass
            elif len(error_nodes) <= 5:
                for n in error_nodes:
                    if n["status"] != "SUCCESS":
                        error_note += f"{n['name']} --> {n['status']}\n"
                    else:
                        error_note += f"{n['name']} --> export {n['export_status']}\n"
            else:
                error_note = "error in more than 5 nodes"
                
            if error_nodes == None:
                pass
            elif len(error_nodes) > 0:
                status = "Failed"
            else:
                status = "Success"

        self.sheet["A" + str(row_id)].fill = PatternFill(start_color='000000', end_color='000000', fill_type='solid')
        self.sheet["B" + str(row_id)] = self.today
        self.sheet["C" + str(row_id)] = datetime.today().strftime("%H:%M")
        self.sheet["F" + str(row_id)] = proj["init_name"]
        self.sheet["G" + str(row_id)] = proj["name"]
        self.sheet["H" + str(row_id)] = proj["loc"]
        self.sheet["I" + str(row_id)] = status
        self.sheet["I" + str(row_id)].fill = self.colors[status]
        self.sheet["J" + str(row_id)] = error_note
        self.sheet["L" + str(row_id)] = "Success" if status=="Success" else ""
        if status == "Success":
            self.sheet["L" + str(row_id)].fill = self.colors[status]
        
        for cell in self.sheet['A'+str(row_id):'L'+str(row_id)][0]:
            cell.border = Border(left=self.side, right=self.side, top=self.side, bottom=self.side)
        
        return error_note, status
        
    def save(self, name):
        self.wb.save(name)

if __name__ == '__main__':
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    load_dotenv()
    username = os.environ.get("USERNAME_DELMAN")
    password = os.environ.get("PASSWORD_DELMAN")
    token = get_token('jovian.yanto@binus.edu', 'Jovian@123')
    excel = ExcelWriter()
    generate_excel(token, excel, proj_file="projects (2).json")

100%|██████████| 45/45 [03:43<00:00,  4.96s/it]
